In [ ]:
import os

# 이 파일과 동일한 폴더에 자신의 gemini API가 텍스트 형태로 담긴 gemini_api.key 파일을 둘 것.
with open('gemini_api.key', 'r') as f:
    api_key = f.read()

os.environ["GOOGLE_API_KEY"] = api_key

In [2]:
from langchain_google_genai import ChatGoogleGenerativeAI

model = ChatGoogleGenerativeAI(model="gemini-1.5-pro-latest")

In [ ]:
# 일반적으로 텍스트 객체가 구분되는 pdf라면 이걸로 읽을 수 있다.
from langchain.document_loaders import PyPDFLoader

# PDF 파일 로드. 파일의 경로 입력
loader = PyPDFLoader("Datasets/초거대_언어모델_연구_동향.pdf")

# 페이지 별 문서 로드
docs = loader.load()

docs[0]

Document(metadata={'source': '초거대_언어모델_연구_동향.pdf', 'page': 0}, page_content='8 특집원고  초거대 언어모델 연구 동향\n초거대 언어모델 연구 동향\n업스테이지  박찬준*･이원성･김윤기･김지후･이활석\n \n1. 서  론1)\nChatGPT1)와 같은 초거대 언어모델(Large Language \nModel, LLM) 의 등장으로 기존에 병렬적으로 연구되\n던 다양한 자연언어처리 하위 분야들이 하나의 모델\n로 처리되고 있으며, 태스크 수렴 현상 (Converge)이 \n발생하고 있다. 즉 하나의 LLM으로 번역, 요약, 질의\n응답, 형태소분석 등의 작업을 모두 처리할 수 있게 \n되었다. 프롬프트 (Prompt)를 어떻게 모델에게 입력하\n느냐에 따라서 LLM의 다양한 능력들이 창발되고, 이\n에 따라 사용자의 목적에 맞는 출력을 생성하는 패러\n다임을 맞이하게 되었다 [1].\nLLM은 최근 몇 년 간의 연구 동향에 따라 뛰어난 \n발전을 이루고 있다. 이러한 발전은 몇 가지 주요한 \n요인에 기반하고 있으며, 이 요인들은 현대 자연언어\n처리 (Natural Language Processing, NLP) 연구의 핵심\n적인 추세로 간주된다. 첫째로, 데이터의 양적 확대는 \n무시할 수 없는 중요한 요인이다. 디지털화의 선도로, \n텍스트 데이터의 양이 기하급수적으로 증가하였고, \n이는 연구의 질적 변화를 가져왔다. 대규모 코퍼스의 \n활용은 LLM의 일반화 능력을 향상시키며, 다양한 맥\n락과 주제에 대한 깊은 학습을 가능하게 한다. 둘째\n로, 컴퓨팅 기술의 진보는 LLM의 발전에 있어 결정\n적이었다. 특히, Graphics Processing Unit (GPU) 및 \nTensor Processing Unit (TPU) 와 같은 고성능 병렬 처\n리 하드웨어의 개발은 모델 학습에 있어 병목 현상을 \n크게 완화시켰다. 이로 인해 연구자들은 모델의 복잡\n성을 키우고, 더욱 깊은 신경망 구

In [ ]:
# # pdf가 스캔본 등의 이유로 텍스트가 아닌 이미지가 나열된 형태의 파일이라면, 아래의 방법으로 실행한다.
# import easyocr
# reader = easyocr.Reader(['ko','en'])
# docs = []  # 페이지 당 인식된 텍스트를 담을 리스트다.
# file_name = '2024_세금절약_가이드.pdf'  # 파일 이름을 적는다.

# # pdf2image에서 covert_from_path 함수 호출
# from pdf2image import convert_from_path
# import numpy as np  # 이미지를 넘파이 배열로 변환하기 위해 로드함.

# # convert_from_path를 이용해 pdf파일를 이미지형태로 불러온다.
# images = convert_from_path('Datasets/' + file_name)

# from langchain_core.documents import Document

# # images는 여러 페이지로 구성되어 있어 아래와 같이 각각의 페이지(이미지)에 대해 OCR을 수행한다.
# for i, image in enumerate(images):
#     docs.append(Document(metadata={'source': file_name, 'page': i}, page_content=' '.join(reader.readtext(np.array(image), detail=0))))

# docs

### 텍스트 분리하기 (Chunking)
단어를 벡터로 나타내는 임베딩을 진행하기 전, 큰 텍스트를 작은 단위로 분리하는 작업을 청킹이라고 하고, <br>
이로 인해 나눠진 작은 조각 단위를 청크(덩어리)라고 한다.

특히 문서가 너무 길어서 벡터로 변환하기 어려울 때, <br>
이 방법을 통해 문서의 의미를 최대한 보존하면서 작은 조각으로 나누어 처리하도록 한다. <br>
임베딩하고자 하는 문서의 종류와 글의 목적 등에 따라 동일한 청킹 기법을 적용해도 성능이 달라질 수 있다.

만일 청크의 크기가 너무 작으면 문맥이 부족할 수 있고, 반대로 너무 크면 검색의 정확도가 감소할 수도 있으므로, <br>
적절한 청크 크기 및 기법을 선택하는 것이 중요하다.

아래는 텍스트를 덩어리로 쪼개는 두 가지 방식을 나열한 것이다.

우선 청킹을 기본적으로 할 수 있는 CharacterTextSplitter 클래스를 보자. 들어가는 인자는 다음과 같다.
- separator: 텍스트를 나눌 기준이 되는 문자열을 지정한다. 만약 이 인자가 없다면, 분할 작업을 하지 않는 것으로 보인다.
- chunk_size: 청크의 길이를 정한다. <br>
위의 seperator로 분할이 가능한 지점 중 chunk_size를 넘지 않는 최대 크기를 갖도록 분할이 진행된다.
- chunk_overlap: 청크 사이에 중복으로 포함할 문자 수를 지정할 수 있다. <br>
이 경우, 이전 청크의 끝 부분 N개의 글자가 다음 청크의 첫 부분 N개의 글자로 등장하게 된다. <br>
중복으로 포함할 문자도 역시 구분자로 구분되어야 하며, 구분자 사이에 있는 문자 개수가 chunk_overlap 미만일 때 위와 같은 효과가 적용된다.
- length_function: 청크의 길이를 계산하는 함수 (대부분 파이썬 내장함수인 len을 쓴다.)
- is_separator_regex: 구분자가 정규식인지를 나타내는 Boolean 값이다.

In [5]:
from langchain.text_splitter import CharacterTextSplitter

text_splitter = CharacterTextSplitter(
    separator="\n",
    chunk_size=100,
    chunk_overlap=30,
    length_function=len,
    is_separator_regex=False,
)

splits = text_splitter.split_documents(docs)
splits[:5]  # 앞에 5개만 출력했다.

[Document(metadata={'source': '초거대_언어모델_연구_동향.pdf', 'page': 0}, page_content='8 특집원고  초거대 언어모델 연구 동향\n초거대 언어모델 연구 동향\n업스테이지  박찬준*･이원성･김윤기･김지후･이활석\n \n1. 서  론1)'),
 Document(metadata={'source': '초거대_언어모델_연구_동향.pdf', 'page': 0}, page_content='1. 서  론1)\nChatGPT1)와 같은 초거대 언어모델(Large Language \nModel, LLM) 의 등장으로 기존에 병렬적으로 연구되'),
 Document(metadata={'source': '초거대_언어모델_연구_동향.pdf', 'page': 0}, page_content='던 다양한 자연언어처리 하위 분야들이 하나의 모델\n로 처리되고 있으며, 태스크 수렴 현상 (Converge)이 \n발생하고 있다. 즉 하나의 LLM으로 번역, 요약, 질의'),
 Document(metadata={'source': '초거대_언어모델_연구_동향.pdf', 'page': 0}, page_content='응답, 형태소분석 등의 작업을 모두 처리할 수 있게 \n되었다. 프롬프트 (Prompt)를 어떻게 모델에게 입력하\n느냐에 따라서 LLM의 다양한 능력들이 창발되고, 이'),
 Document(metadata={'source': '초거대_언어모델_연구_동향.pdf', 'page': 0}, page_content='느냐에 따라서 LLM의 다양한 능력들이 창발되고, 이\n에 따라 사용자의 목적에 맞는 출력을 생성하는 패러\n다임을 맞이하게 되었다 [1].')]

아래의 RecursiveCharacterTextSplitter도 비슷하지만, 위와 같이 지정한 문자 기준으로 자르는 것이 아니라, 여러 기준을 순차적으로 적용하며 잘라 나간다. <br>
단락, 문장, 단어 단위 순으로 탐색하면서 분할하며 이렇게 하면 문서의 구조를 유지하기가 쉬워진다. (이 이유로 seperator를 따로 지정하지 않아도 된다.) <br>
여기서는 상기의 이유로 RecursiveCharacterTextSplitter를 사용하도록 하겠다.

In [6]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

recursive_text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=100,
    chunk_overlap=2,
    length_function=len,
    is_separator_regex=False,
)

splits = recursive_text_splitter.split_documents(docs)
splits[:5]

[Document(metadata={'source': '초거대_언어모델_연구_동향.pdf', 'page': 0}, page_content='8 특집원고  초거대 언어모델 연구 동향\n초거대 언어모델 연구 동향\n업스테이지  박찬준*･이원성･김윤기･김지후･이활석\n \n1. 서  론1)'),
 Document(metadata={'source': '초거대_언어모델_연구_동향.pdf', 'page': 0}, page_content='ChatGPT1)와 같은 초거대 언어모델(Large Language \nModel, LLM) 의 등장으로 기존에 병렬적으로 연구되\n던 다양한 자연언어처리 하위 분야들이 하나의 모델'),
 Document(metadata={'source': '초거대_언어모델_연구_동향.pdf', 'page': 0}, page_content='로 처리되고 있으며, 태스크 수렴 현상 (Converge)이 \n발생하고 있다. 즉 하나의 LLM으로 번역, 요약, 질의\n응답, 형태소분석 등의 작업을 모두 처리할 수 있게'),
 Document(metadata={'source': '초거대_언어모델_연구_동향.pdf', 'page': 0}, page_content='되었다. 프롬프트 (Prompt)를 어떻게 모델에게 입력하\n느냐에 따라서 LLM의 다양한 능력들이 창발되고, 이\n에 따라 사용자의 목적에 맞는 출력을 생성하는 패러'),
 Document(metadata={'source': '초거대_언어모델_연구_동향.pdf', 'page': 0}, page_content='다임을 맞이하게 되었다 [1].\nLLM은 최근 몇 년 간의 연구 동향에 따라 뛰어난 \n발전을 이루고 있다. 이러한 발전은 몇 가지 주요한')]

In [7]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings

# gemini의 임베딩 모델을 적용한다. 임베딩은 위에서도 언급했듯 단어를 벡터로 변환하는 과정이다.
embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001") # gemini의 임베딩 모델

### FAISS
- FAISS는 Facebook에서 개발 및 배포한 밀집 벡터의 유사도 측정과 클러스터링에 효율적인 라이브러리다.
- 여기서 from_documents() 메서드를 사용하면 위에서 만든 Document 객체들을 활용하여 FAISS 벡터 저장소를 생성한다.
- 이후 이렇게 저장된 벡터들을 활용하여 유사도를 측정하거나 이를 활용한 비슷한 유사도를 갖는 벡터를 검색하는 일에 활용한다.

In [8]:
import faiss
from langchain_community.vectorstores import FAISS

vector_store = FAISS.from_documents(documents=splits, embedding=embeddings)

벡터 저장소 객체의 메서드인 as_retriever()는 현재 벡터 저장소를 기반으로 VectorStoreRetriever 객체를 생성하도록 한다.
- VectorStoreRetriever 객체는 벡터 저장소를 기반으로 하는 검색기 객체이다.
- search_type은 유사한 벡터를 찾는 기준이며 여기서는 기본값인 similarity(유사도)를 사용한다.
- search_kwargs는 검색 함수에 전달할 추가 키워드 인자로 다음과 같은 것들이 있다. <br><br>
    - k: 반환할 문서 수
    - score_threshold: 유사도 점수 임계값
    - fetch_k: MMR 알고리즘에 전달할 문서 수
    - lambda_mult: MMR 다양성 조절 파라미터
    - filter: 문서 메타데이터 기반 필터링

여기서는 반환할 문서 수인 k값을 조정해 준다. fetch_k와 k값을 적절히 조정하여 최적의 성능을 나오도록 할 수 있다.

In [9]:
retriever = vector_store.as_retriever(search_type="similarity", search_kwargs={"k": 3})

In [10]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough

# 프롬프트 템플릿 정의
contextual_prompt = ChatPromptTemplate.from_messages([
    ("system", "반드시 이미 주어져 있는 맥락만을 기반으로 해서 한국어로 답변해 줘. 또한 주의사항은 말하지 마."),
    ("user", "Context: {context}\\n\\nQuestion: {question}")
])

In [11]:
class DebugPassThrough(RunnablePassthrough):
    def invoke(self, *args, **kwargs):
        output = super().invoke(*args, **kwargs)
        print("Debug Output:", output)
        return output
# 문서 리스트를 텍스트로 변환하는 단계 추가
class ContextToText(RunnablePassthrough):
    def invoke(self, inputs, config=None, **kwargs):  # config 인수 추가
        # context의 각 문서를 문자열로 결합
        context_text = "\n".join([doc.page_content for doc in inputs["context"]])
        return {"context": context_text, "question": inputs["question"]}

# RAG 체인에서 각 단계마다 DebugPassThrough 추가
rag_chain_debug = {
    "context": retriever,                    # 컨텍스트를 가져오는 retriever
    "question": DebugPassThrough()        # 사용자 질문이 그대로 전달되는지 확인하는 passthrough
}  | DebugPassThrough() | ContextToText()|   contextual_prompt | model


In [ ]:
# 이제 gemini와 연결해서 대화를 진행해보자.
while 1: 
	print("========================")
	query = input("질문을 입력하세요: ")
	response = rag_chain_debug.invoke(query)
	print("Final Response:")
	print(response.content)

Debug Output: 이 논문에는 무슨 내용이 있는가?
Debug Output: {'context': [Document(metadata={'source': '초거대_언어모델_연구_동향.pdf', 'page': 7}, page_content='등의 내재적인 한계를 지니고 있다. 또한, 자연어 코\n퍼스를 활용하여 학습되기 때문에, 주요 NLP 태스크\n가 아닌 산술 추론 (e.g., 1234+4321=?) 등에 약점을'), Document(metadata={'source': '초거대_언어모델_연구_동향.pdf', 'page': 4}, page_content='연구는 아직 초기 단계에 머물러 있다. 이와 관련하여 \n주목할 만한 연구로는 [41]이 있다. 해당 연구에서는 \n사전 학습 코퍼스를 시간대, 필터링 기법, 도메인 혼'), Document(metadata={'source': '초거대_언어모델_연구_동향.pdf', 'page': 6}, page_content='프롬프트는 자연어 텍스트 형태의 태스크 설명, 시연\n을 위한 몇 가지 예시 및 테스트 쿼리로 구성된다. 최\n신 연구 [82]에 따르면, ICL 은 다음과 같은 다양한 이')], 'question': '이 논문에는 무슨 내용이 있는가?'}
Final Response:
자연어 처리(NLP) 모델은 내재적인 한계를 가지고 있으며, 특히 산술 추론(e.g., 1234+4321=?)에 약하다. 이러한 한계를 극복하기 위한 연구는 아직 초기 단계이다.  [41]에서는 사전 학습 코퍼스를 시간대, 필터링 기법, 도메인 등으로 나누어 학습하는 방법을 제시했고,  [82]에서는 In-Context Learning(ICL)에 대해 다루고 있다. ICL은 자연어 텍스트 형태의 태스크 설명, 시연 예시, 테스트 쿼리로 구성된 프롬프트를 사용한다.

Debug Output: 혹시 자연어 처리 모델이 무엇인지 설명해 줄 수 있나요?
Debug Output: {'context': [Document(metadata={'s